In [8]:
import numpy as np
import scipy.linalg as sci
np.set_printoptions(precision=3,suppress=True)

# Hessenberg Decomposition

In [16]:
def HH(A):
    n = np.size(A,axis=0)
    Q = np.eye(n)
              
    for k in range(0,n-2):
        x = A[k+1:,k]
        lx = len(x)

        p = (-np.sign(x[0])*np.linalg.norm(x))
        v = -x 
        v[0] = v[0] + p
        R = np.eye(lx) - 2*(np.outer(v,v))/(v.T@v)
        
        Qi = sci.block_diag(np.eye(k+1),R)
        A = Qi@A@Qi.T                                #This is the recursion to get the Hessenberg matrix
        Q = Q@Qi                                     #This is the recursion to get the orthogonal matrix

    H=A    
    return H,Q


# Givens Rotation

In [17]:
def GivensRot(a,b):                  #Code for Givens Rotation
    givens_mat = np.zeros((2,2))
    
    if abs(b) >= abs(a):
        t = a/b
        s = 1 / ((1+t**2)**0.5)
        c = s*t
        
    elif abs(a) >= abs(b):
        t = b/a
        c = 1 / ((1+t**2)**0.5)
        s = c*t
        
    return np.array([[c,-s],[s,c]])   #Output is the 2x2 rotation matrix

          
def GivensQRHess(H):                  #QR Decomposition using Givens Rotation
    n = np.size(H,axis=0)
    Q = np.eye(n)
    
    for k in range(0,n-1):
        a = H[k,k]
        b = H[k+1,k]
        giv_mat = GivensRot(a,b)
        
        G = sci.block_diag(np.eye(k),giv_mat,np.eye(n-k-2))
        H = G.T@H
        Q = Q@G
    
    return H,Q                       #Output is the new Hessenberg matrix H and orthogonal matrix Q


# Upper Hessenberg Matrix Converger

In [26]:
def ConvergeHess(H,k):      #Input an upper Hessenberg matrix for H and the number of times k you want 
                            #to run the Givens QR decomposition code
    if (k == 0):
        return H
    else:
        Hk = H
        n = 0 
        while (n <= k):
            Ht,Qt = GivensQRHess(Hk)
            Hk = Ht
            n += 1
    return Hk               #Output will be upper triangular or Hessenberg depending on k


# Example of Hessenberg Matrix Converger

In [21]:
ex_mat = np.array([[4.8574,0.2750,-1.0182,0.6499,-1.0429,1.1356,-0.2263,0.0595],
                       [0.3412,3.2061,-0.3560,-0.1288,-0.5041,0.1073,-1.1366,-0.6932],
                       [-0.9537,-0.3791,3.5085,0.5173,1.2230,-1.1862,1.2539,-0.6567],
                       [0.7633,-0.1537,0.5710,5.5440,-1.1054,-0.6215,-0.0770,-0.3176],
                       [-1.0826,-0.4345,1.2816,-1.0685,6.2123,0.7262,-0.8017,0.5979],
                       [1.1230,0.1449,-1.2203,-0.6752,0.8096,4.1445,0.2589,-0.8535],
                       [-0.0955,-1.1556,1.1972,-0.1305,-0.8919,0.3975,5.5447,-0.4539],
                       [0.0208,-0.6059,-0.7259,-0.3221,0.5742,-0.8451,-0.4686,2.9769]])


In [22]:
H0,Q0 = HH(ex_mat)

H0 #Upper Hessenberg matrix of ex_mat

array([[ 4.857, -1.981, -0.025,  0.093,  0.11 ,  0.053,  0.113,  0.061],
       [-2.013,  5.717, -1.506, -0.106, -0.056,  0.021, -0.086,  0.06 ],
       [ 0.   , -1.519,  4.813, -1.905, -0.06 , -0.073,  0.029,  0.061],
       [-0.   , -0.   , -1.906,  3.47 ,  1.897,  0.019,  0.028, -0.085],
       [ 0.   ,  0.   , -0.   ,  1.95 ,  4.755,  1.315,  0.044, -0.08 ],
       [ 0.   , -0.   ,  0.   ,  0.   ,  1.255,  3.513,  1.48 , -0.073],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.534,  4.147, -1.227],
       [-0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   , -1.18 ,  4.723]])

In [23]:
ConvergeHess(H0,10) 

array([[ 5.258, -4.019,  0.553,  0.127,  0.123,  0.041,  0.138,  0.033],
       [ 0.   ,  4.772, -2.86 ,  0.548,  0.01 ,  0.061, -0.044,  0.055],
       [ 0.   ,  0.   ,  4.536, -3.115, -0.851, -0.06 ,  0.003,  0.111],
       [ 0.   , -0.   ,  0.   ,  3.078,  4.325,  0.828,  0.053, -0.083],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  2.891,  2.446,  0.655, -0.064],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  3.123,  3.193, -0.647],
       [-0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,  3.187, -2.718],
       [-0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  4.001]])

In [24]:
ConvergeHess(H0,50) 

array([[ 5.258, -4.019,  0.553,  0.127,  0.123,  0.041,  0.138,  0.033],
       [ 0.   ,  4.772, -2.86 ,  0.548,  0.01 ,  0.061, -0.044,  0.055],
       [ 0.   ,  0.   ,  4.536, -3.115, -0.851, -0.06 ,  0.003,  0.111],
       [ 0.   , -0.   ,  0.   ,  3.078,  4.325,  0.828,  0.053, -0.083],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  2.891,  2.446,  0.655, -0.064],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  3.123,  3.193, -0.647],
       [-0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,  3.187, -2.718],
       [-0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  4.001]])

In [25]:
ConvergeHess(H0,100) 

array([[ 5.258, -4.019,  0.553,  0.127,  0.123,  0.041,  0.138,  0.033],
       [ 0.   ,  4.772, -2.86 ,  0.548,  0.01 ,  0.061, -0.044,  0.055],
       [ 0.   ,  0.   ,  4.536, -3.115, -0.851, -0.06 ,  0.003,  0.111],
       [ 0.   , -0.   ,  0.   ,  3.078,  4.325,  0.828,  0.053, -0.083],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  2.891,  2.446,  0.655, -0.064],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  3.123,  3.193, -0.647],
       [-0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,  3.187, -2.718],
       [-0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  4.001]])